In [1]:
import json 
import pandas as pd

In [2]:
d = []
for l in open('review.json').readlines():
    d.append(json.loads(l)) # from json str to dict
df = pd.DataFrame.from_records(d[0:1000000])[['user_id', 'text']]

In [3]:
from textblob import TextBlob
df['sentiment'] = df[['text']].applymap(lambda x: TextBlob(x).sentiment.polarity) > 0
df['sentiment'] = df['sentiment'] * 1 # 1 for positive review, 0 for negative

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
X = df['user_id']
y = df['sentiments']
logreg = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=350).fit(X, y)
score = cross_val_score(LogisticRegression(solver='lbfgs', multi_class='auto'), X, y, cv=5, scoring='roc_auc')
score.mean()

KeyError: 'sentiments'

In [6]:
data=[]
for l in open("user.json").readlines():
    data.append(json.loads(l))
df = pd.DataFrame.from_records(data)
df.dropna(subset=['elite', 'friends', 'review_count', 'yelping_since'])
elite = df['elite'].str.split(',')
elite = elite.map(lambda x : 0 if x[0] == '' else len(x))
df['elite'] = elite
friends = df['friends'].str.split(',')
friends = friends.map(lambda x : 0 if x[0] == '' else len(x))
df['friends'] = friends
df['yelping_since'] = df['yelping_since'].str.slice(0,4)
df['yelping_since'] = pd.to_numeric(df['yelping_since'], errors='coerce')

In [7]:
X = df[['elite', 'yelping_since']]
y = df['friends']
from sklearn.ensemble import GradientBoostingRegressor
CV_score = cross_val_score(GradientBoostingRegressor(), X, y, cv=5,scoring = 'r2')
CV_score

array([0.17081617, 0.12358706, 0.08631497, 0.06571231, 0.04685467])

In [8]:
CV_score.mean()

0.09865703620303583